In [9]:
# import statements
import json
import os
import time
import re

In [2]:
# Define the keywords that will be used to extract the imperative sentences
# Rules are identified from the paper - https://www.aclweb.org/anthology/W14-2117.pdf
# 1. "find those sentences with a verb (in its base form) as the root 
#     in the phrase structure and this particular verb has no 
#.    subject child in the dependency structure" 
#.    Example: You must first discuss the matter there, and you need to be specific”
# 2. "recognize the use of a personal pronoun or noun (e.g., “you”, “we”,
#.     or a username) followed by a modal verb (e.g., “should”, “must”, “need”) as an imperative"


# Source: https://www.myenglishpages.com/site_php_files/grammar-lesson-modals.php
modal_verbs = ['can', 'could', 'may', 'might', 'will', 'would', 'shall','should', 'must', 'ought', 'dare', 'had better', 'ask']
modal_verbs = ['can', 'may', 'might', 'will', 'would', 'shall','should', 'must', 'ought', 'dare', 'had better', 'ask']

# Source: https://grammar.yourdictionary.com/parts-of-speech/pronouns/list-of-personal-pronouns.html
# personal_pronoun = ['I', 'me', 'we', 'us', 'you', 'he', 'she', 'her', 'him', 'it', 'they', 'them']
personal_pronoun = ['us', 'you', 'he', 'she', 'her', 'him', 'it', 'they', 'them']

In [3]:
# Check for rule 2

# Create combinations of possibilities for a sentence to be imperative
combinations = []
# form combinations of words and put them as a list
for pronoun in personal_pronoun:
    for verb in modal_verbs:
        combinations.append(pronoun + ' ' + verb)
        
print('Number of combinations for rule 2 =', len(combinations))

Number of combinations for rule 2 = 108


In [4]:
# Using rule 2 to check imperative snetences
def check_imperative_r2(combinations, sentence):
    if any(combo in sentence.lower() for combo in combinations):
        return True
    return False

In [5]:
IP_DIR = '../data/input/'
OP_DIR = '../data/output/'

dataset = 'friends/'

## Friends Dialogue Dataset

In [6]:
# Renaming files
ip_path = os.path.join(IP_DIR, dataset)
op_path = os.path.join(OP_DIR, dataset)

text_key = 'text'
op_ext = '.friends'
op_filebase = 'dialogues_'

sentence = []
count = 0
file_count = 0

tic = time.time()

files = os.listdir(ip_path)
# print('Files to be processed =', len(tweets_files))

for filename in files:
    if not filename.endswith('.friends'): 
        old_name = os.path.join(ip_path, filename)
        new_name = os.path.join(ip_path, filename + '.friends')
        os.rename(old_name, new_name)
        # print(old_name, new_name)

In [7]:
def check_for_char_name(text):
    text = text.strip().lower()
    if (re.match('^[a-z]', text[0])):
        return True
    return False

In [ ]:
# Renaming files
ip_path = os.path.join(IP_DIR, dataset)
op_path = os.path.join(OP_DIR, dataset)

text_key = 'text'
op_ext = '.friends'
op_filebase = 'dialogues_'
delimiter = ':'

sentence = []
count = 0
file_count = 0
sent_per_file = 100000

tic = time.time()

files = os.listdir(ip_path)

for filename in files:
    
    if filename.endswith('.friends'): 
        
        ip_filename = os.path.join(ip_path, filename)
        with open(ip_filename, "r") as fp:
            
            for line in fp:
                line = line.strip()
                if (len(line) > 0):
                    
                    split_text = line.split(delimiter, 1)
                    
                    if (len(split_text) == 2):
                        
                        if check_for_char_name(split_text[0]):
                            character = split_text[0].strip()
                            text = split_text[1].strip()
                            
                            print(character, text)
                            
                            if ((count%sent_per_file) == 0):
                                op_filename = op_filebase+str(count+1)+op_ext
                                op_filepath = os.path.join(op_path, op_filename)
                                
                                with open(op_filepath, 'w', encoding='utf-8') as fp:
                                    fp.write(text+'\n')
                            else:
                                with open(op_filepath, 'a', encoding='utf-8') as fp:
                                    fp.write(text+'\n')

                    # sentence.append(text)
                    
                            count += 1
    file_count += 1
    print('Files ('+str(file_count)+'/'+str(len(files))+') - Tweets = ' + str(count) \
            + ' - File = ' + op_filename, end='\r')
    
print('\nTotal time:', (time.time()-tic), 'seconds')

# Rule 2

In [12]:
ip_path = os.path.join(OP_DIR, dataset)
op_filename = os.path.join(OP_DIR, 'imperative_rule2.txt')

tic = time.time()

processed_files = os.listdir(ip_path)
print('Files to be processed =', len(processed_files))

imper_t = []
imper_count = 0
sent_count = 0
file_count = 0

with open(op_filename, 'w', encoding='utf-8') as op:
    for filename in processed_files:
        if filename.endswith('.friends'): 
            ip_filename = os.path.join(ip_path, filename)
            
            with open(ip_filename, 'r', encoding='utf-8') as ip:

                for sentence in ip:
                    if check_imperative_r2(combinations, sentence):
                        imper_t.append(sentence)
                        op.write(sentence+'\n')

                        imper_count += 1
                    sent_count += 1

            file_count += 1
        print('Files processed ('+str(file_count)+'/'+str(len(processed_files))+ \
                      ') - Total imperatives = ' + str(len(imper_t)) + '/' + str(sent_count), end='\r')

print('\nTotal time:', (time.time()-tic), 'seconds')

Files to be processed = 1
Files processed (1/1) - Total imperatives = 788/23264
Total time: 0.9597980976104736 seconds


In [13]:
imper_t

["Yu-huh. That's when my mother would shut off the TV and say 'The end'.\n",
 "Yeah, Ross can't go so it's between my friend Eric Prower who has breath issues and Dan with the poking. [starts poking Monica in the shoulder] 'Did you see that play? Do you want some more beer? Is that Spike Lee?'\n",
 "Ok. [Richard walks in] Hey, why don't you ask Richard?\n",
 "Wow. Well being a huge Knicks fan myself, I think you should take someone who's a huge Knicks fan.\n",
 "So what, he's a little older, big deal, I mean he's important to me. Ya know if you ask him, he might take you on his Jag. [walks off]\n",
 'You can watch our tape if you want.\n',
 "Actually, he is getting closer on the talking thing. He can't quite say mama yet, but once he said yumen.\n",
 'Wow, he must like you the best.\n',
 'Well for starters, you may want to light it and lose the spatula.\n',
 "Hi, we're visitiing. It's Ben and his da-da. Da-da. Can you say da-da? Look, I'm gonna tell your momies you said it anyway so yo

# Rule 1

In [14]:
#load libraries and environments
import spacy
from spacy import displacy

In [15]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [16]:
#rule 1: Has a verb in it's lemma (base) form
def verb_in_lemma(sent):
    doc = nlp(sent)
    for token in doc:
        if token.pos_=='VERB' and token.text.lower() == token.lemma_:
            return True

In [17]:
#rule 2: Has a verb in its lemma (base) form and is the root
def root_verb_in_lemma(sent):
    doc = nlp(sent)
    for token in doc:
        if token.dep_=='ROOT' and token.pos_=='VERB' and token.text.lower() == token.lemma_:
            print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            return True

In [18]:

#rule 3: Has a verb in its lemma (base) form and is the root and does not have any subject child in it's dependency structure
def root_verb_in_lemma_nosubj(sent):
    doc = nlp(sent)
    cond1 = False
    cond2 = True
    for token in doc:
        if token.dep_=='ROOT' and token.pos_=='VERB' and token.text.lower() == token.lemma_:
            cond1 = True
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
        if token.dep_ == 'nsubj':
            cond2 = False   
    return (cond1 and cond2)

In [19]:
ip_path = os.path.join(OP_DIR, dataset)
op_filename = os.path.join(OP_DIR, 'imperative_rule1.txt')

tic = time.time()

processed_files = os.listdir(ip_path)
print('Files to be processed =', len(processed_files))

imper_t = []
imper_count = 0
sent_count = 0
file_count = 0

with open(op_filename, 'w', encoding='utf-8') as op:
    for filename in processed_files:
        if filename.endswith('.friends'): 
            ip_filename = os.path.join(ip_path, filename)
            
            with open(ip_filename, 'r', encoding='utf-8') as ip:

                for sentence in ip:
                    if root_verb_in_lemma_nosubj(sentence):
                        imper_t.append(sentence)
                        op.write(sentence+'\n')

                        imper_count += 1
                    sent_count += 1

            file_count += 1
        print('Files processed ('+str(file_count)+'/'+str(len(processed_files))+ \
                      ') - Total imperatives = ' + str(len(imper_t)) + '/' + str(sent_count), end='\r')

print('\nTotal time:', (time.time()-tic), 'seconds')

Files to be processed = 1
Files processed (1/1) - Total imperatives = 720/23264
Total time: 206.4490668773651 seconds


In [20]:
imper_t

['Thank you.\n',
 'Ok, here, watch this.\n',
 'Oh, then go Vassar.\n',
 'Ok, then just go.\n',
 'Thank you.\n',
 'Kick save and... denied.\n',
 'Take care.\n',
 'Let’s do it! Ross?\n',
 'To score the winning touchdown, by the way.\n',
 'Okay. Let’s bring it in.\n',
 'Thank you.\n',
 'Sweetie, don’t worry you’ll get picked. Chandler.\n',
 'Wow! Just like in the pros.\n',
 'Huddle up.\n',
 '(to his team) All right, huddle up, right over here.\n',
 'Break.\n',
 'Oww!! Son of a...!! Ow! Come on!\n',
 'Go! Go! Go!\n',
 'Score!! 7 to nothing!\n',
 'Wait, how long?\n',
 'Break!!\n',
 'Set....hike!\n',
 'Okay. Hut! Hike!\n',
 'Fumble!\n',
 'Touchdown!! Touchdown!!\n',
 'Does so count!\n',
 'Break!\n',
 'Whoa, whoa, no, no, I-I’m not playing with this guy, now.\n',
 'Yeah!!! Kill ‘um!!!\n',
 'Thank you! Break!\n',
 'Thirty-two! Seventy-one! Hike!\n',
 'Break!\n',
 'Let go! Let go!\n',
 'Let go!\n',
 'Gimme the this!\n',
 'Let go!\n',
 'Thank you.\n',
 'So the ah, play’s pretty great, huh?\n',
 